In [1]:
# pip install tensorflow

  Using cached tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata (3.6 kB)
  Using cached tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata (5.1 kB)
Using cached tensorflow-2.15.0-cp311-cp311-win_amd64.whl (2.1 kB)
Using cached tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl (300.9 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np


In [3]:
data = pd.read_csv('train.csv')

data['Date'] = pd.to_datetime(data['Date'])

data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day

data.drop(['Date'], axis=1, inplace=True)

data_cleaned = data.dropna(subset=['Task'])


In [4]:
category0 = data[data['Task'] == 0]
category1 = data[data['Task'] == 1]

diff = len(category0) - len(category1)
class_0_size = len(category0)
class_1_size = len(category1)
diff

num_rows_to_drop = int((class_0_size - len(data)/2) )

rows_to_drop = category0.sample(n=num_rows_to_drop, random_state=42)

category0 = category0.drop(rows_to_drop.index)

category0.reset_index(drop=True, inplace=True)
category0['Task'].value_counts()

diff = len(category0) - len(category1)
class_0_size = len(category0)
class_1_size = len(category1)
diff

category1_oversampled = category1.sample(n=class_0_size, replace=True,random_state=42)

data = pd.concat([category0, category1_oversampled])
data['Task'].value_counts()

Task
0.0    58184
1.0    58184
Name: count, dtype: int64

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

data_cleaned = data

X = data_cleaned.drop('Task', axis=1)
y = data_cleaned['Task']

categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(exclude=['object']).columns

numerical_imputer = SimpleImputer(strategy='mean')
X[numerical_cols] = numerical_imputer.fit_transform(X[numerical_cols])

categorical_imputer = SimpleImputer(strategy='most_frequent')
X[categorical_cols] = categorical_imputer.fit_transform(X[categorical_cols])

C:\Users\achin\anaconda3\envs\WeatherClassification\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\achin\anaconda3\envs\WeatherClassification\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\achin\anaconda3\envs\WeatherClassification\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\achin\anaconda3\envs\WeatherClassification\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse i

In [6]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numerical_transformer = StandardScaler()

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

X_train_preprocessed = X_train_preprocessed.toarray()
X_test_preprocessed = X_test_preprocessed.toarray()

C:\Users\achin\anaconda3\envs\WeatherClassification\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\achin\anaconda3\envs\WeatherClassification\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\achin\anaconda3\envs\WeatherClassification\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\achin\anaconda3\envs\WeatherClassification\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse i

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

In [9]:
model = keras.Sequential([
    keras.layers.Dense(256, activation='relu', input_shape=(X_train_preprocessed.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(192, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(160, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train_preprocessed,
    y_train,
    epochs=150,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=2
)

test_loss, test_acc = model.evaluate(X_test_preprocessed, y_test)
print(f'Test accuracy: {test_acc}')





Epoch 1/150


1164/1164 - 5s - loss: 0.4768 - accuracy: 0.7751 - val_loss: 0.4334 - val_accuracy: 0.7956 - 5s/epoch - 4ms/step
Epoch 2/150
1164/1164 - 3s - loss: 0.4437 - accuracy: 0.7960 - val_loss: 0.4203 - val_accuracy: 0.8061 - 3s/epoch - 3ms/step
Epoch 3/150
1164/1164 - 3s - loss: 0.4279 - accuracy: 0.8046 - val_loss: 0.4085 - val_accuracy: 0.8087 - 3s/epoch - 3ms/step
Epoch 4/150
1164/1164 - 3s - loss: 0.4173 - accuracy: 0.8098 - val_loss: 0.4035 - val_accuracy: 0.8112 - 3s/epoch - 3ms/step
Epoch 5/150
1164/1164 - 3s - loss: 0.4083 - accuracy: 0.8141 - val_loss: 0.3973 - val_accuracy: 0.8163 - 3s/epoch - 3ms/step
Epoch 6/150
1164/1164 - 3s - loss: 0.4009 - accuracy: 0.8188 - val_loss: 0.3903 - val_accuracy: 0.8188 - 3s/epoch - 3ms/step
Epoch 7/150
1164/1164 - 3s - loss: 0.3952 - accuracy: 0.8201 - val_loss: 0.3861 - val_accuracy: 0.8241 - 3s/epoch - 3ms/step
Epoch 8/150
1164/1164 - 3s - loss: 0.3876 - accuracy: 0.8245 - val_loss: 0.3828 - val_accuracy: 0.8230 - 3s/epoch - 3ms/s

Epoch 59/150
1164/1164 - 6s - loss: 0.2896 - accuracy: 0.8781 - val_loss: 0.3390 - val_accuracy: 0.8569 - 6s/epoch - 5ms/step
Epoch 60/150
1164/1164 - 5s - loss: 0.2898 - accuracy: 0.8777 - val_loss: 0.3371 - val_accuracy: 0.8560 - 5s/epoch - 4ms/step
Epoch 61/150
1164/1164 - 4s - loss: 0.2875 - accuracy: 0.8778 - val_loss: 0.3379 - val_accuracy: 0.8569 - 4s/epoch - 4ms/step
Epoch 62/150
1164/1164 - 4s - loss: 0.2863 - accuracy: 0.8792 - val_loss: 0.3403 - val_accuracy: 0.8594 - 4s/epoch - 3ms/step
Epoch 63/150
1164/1164 - 4s - loss: 0.2859 - accuracy: 0.8784 - val_loss: 0.3392 - val_accuracy: 0.8585 - 4s/epoch - 3ms/step
Epoch 64/150
1164/1164 - 5s - loss: 0.2845 - accuracy: 0.8800 - val_loss: 0.3420 - val_accuracy: 0.8581 - 5s/epoch - 4ms/step
Epoch 65/150
1164/1164 - 4s - loss: 0.2873 - accuracy: 0.8796 - val_loss: 0.3373 - val_accuracy: 0.8599 - 4s/epoch - 4ms/step
Epoch 66/150
1164/1164 - 4s - loss: 0.2836 - accuracy: 0.8810 - val_loss: 0.3347 - val_accuracy: 0.8590 - 4s/epoch - 3

In [10]:

new_data = pd.read_csv('test.csv')


new_data['Date'] = pd.to_datetime(new_data['Date'])

new_data['Year'] = new_data['Date'].dt.year
new_data['Month'] = new_data['Date'].dt.month
new_data['Day'] = new_data['Date'].dt.day

new_data.drop(['Date'], axis=1, inplace=True)

ids = new_data["ID"]
new_data.drop("ID", axis = 1, inplace = True)

numerical_cols = new_data.select_dtypes(exclude=['object']).columns
categorical_cols = new_data.select_dtypes(include=['object']).columns

numerical_imputer = SimpleImputer(strategy='mean')
new_data[numerical_cols] = numerical_imputer.fit_transform(new_data[numerical_cols])

categorical_imputer = SimpleImputer(strategy='most_frequent')
new_data[categorical_cols] = categorical_imputer.fit_transform(new_data[categorical_cols])

X_new = new_data

X_new_preprocessed = preprocessor.transform(X_new)

X_new_preprocessed = X_new_preprocessed.toarray()

predictions = model.predict(X_new_preprocessed)

rounded_predictions = np.round(predictions)




C:\Users\achin\anaconda3\envs\WeatherClassification\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\achin\anaconda3\envs\WeatherClassification\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\achin\anaconda3\envs\WeatherClassification\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\achin\anaconda3\envs\WeatherClassification\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse i

889/889 [==============================] - 2s 2ms/step


In [11]:
dict = {'ID':[],
        'Task':[],
       }
for i in range(new_data.shape[0]):
    dict['ID'].append(ids.iloc[i])
    dict['Task'].append(int(rounded_predictions[i]))

ans = pd.DataFrame(dict)

ans.to_csv("subded3.csv", index=False)

C:\Users\achin\AppData\Local\Temp\ipykernel_25316\1741064945.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dict['Task'].append(int(rounded_predictions[i]))
